# Spam or ham?

In the spam literature, an email that is **not** spam is called _ham_. 

Your task is to develop a naïve Bayes classifier to determine whether a given email is spam or ham.

The following cell loads your training data set (called _training set_).

In [1]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam (`1.`) or ham (`0.`). The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value `1.` if the keyword appears in the message and `0.` otherwise.

## The model:  naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* **$C = 1$ for spam messages **
* **$C = 0$ for ham messages. **


The classifier builds a model for the probability $P(C=c\ |\ message)$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate
$\require{color}$
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \  \textcolor{blue}{P(C=c\ |\ message)}.
\end{equation}
Using Bayes' rule we can write

\begin{equation}
P(C=c\ |\ message) = \frac{P(message\ |\ C=c)P(C=c)}{P(message\ |\ C=1)P(C=1) + P(message\ |\ C=0)P(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
\textcolor{blue}{P(C=c\ |\ message)} \propto \textcolor{orange}{P(message\ |\ C=c)}\textcolor{green}{P(C=c)},
\end{equation}

where $\propto$ means "proportional to". The class priors $\textcolor{green}{P(C=c)}$ can be computed directly (you will do so in exercise A) but we need to further simplify $\textcolor{orange}{P(message\ |\ C=c)}$.


### Choice of the event model: _Multinomial_ naïve Bayes

Different naïve Bayes models differ in their distributional assumptions about $\textcolor{orange}{P(message\ |\ C=c)}$. We represent a message using a **binary** [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as a set of $k$ keywords, that is, $message = (w_1, ..., w_k)$, where $w_i = 1$ if the  keyword $w_i$ appears in the message and $w_i = 0$ otherwise.

We assume that the $P(w_1, ..., w_k |\ C=c)$ follows a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) for each class. Note that this event model is slightly different from the one used in the lecture, where we assumed a [multivariate Bernoulli event model](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Parameter_estimation_and_event_models). Intuitively, the multinomial distribution assumes that the words of a message were "drawn" independently from a bag of $k$ different words. Depending on the class membership $c$, each keyword $w$ has a probability $\theta_{c, w}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy,... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment,... $\}$, if the training data was mostly gathered from emails of researchers.

Under these assumptions, the likelihood of a message, given that it belongs to class $c$, is then proportional to
\begin{equation}
\textcolor{orange}{P(message\ |\ C=c)} \propto \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i}.
\end{equation}


The parameters $\textcolor{brown}{\theta_{c, w}}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$, that is,
\begin{equation}
\textcolor{brown}{\theta_{c, w}} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $n_{c, w}$ is the number of times the keyword $w$ appears in messages of class $c$ in the training set and $n_{c}$ is the total count of keywords for all messages of class $c$, that is, $n_{c} = \sum_w n_{c, w}$.



We are now finally able to rewrite the *maximum a posteriori* estimate in a form that is easy to compute:
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ \textcolor{green}{P(C=c)}   \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i}].
\end{equation}


#### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ log( \textcolor{green}{P(C=c)}   \prod_{i = 1}^k  (\textcolor{brown}{\theta_{c, w_i}})^{w_i} ) \\
 = \underset{c \in \{0,1\}}{argmax} \ [ log( \textcolor{green}{P(C=c)}) + \sum_{i = 1}^k w_i \ log(\textcolor{brown}{\theta_{c, w_i}}) ].
\end{equation}

## Part A: Estimate class priors (20 marks)

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **logarithm** of the empirical class priors $\textcolor{green}{P(C=c)}$ for $c \in \{0, 1\}$.

Use the function **np.log()** (the natural log) to compute logarithms throughout this notebook. Avoid using all other bases for logarithm computations.

In [2]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
    log(P(C=0)) and log(P(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    ### YOUR CODE HERE...
    zeros = np.count_nonzero(data[:,0]==0)
    ones = np.count_nonzero(data[:,0]==1)
    
    probability_zeros = zeros/1000
    probability_ones = ones/1000
    
    log_zeros = np.log(probability_zeros)
    log_ones = np.log(probability_ones)
    
    log_class_priors = np.array([log_zeros,log_ones])
    return log_class_priors

estimate_log_class_priors(training_spam)

array([-0.48939034, -0.94933059])

In [3]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))


result [-0.48939034 -0.94933059]


## Part B: Estimate class-conditional likelihoods (40 marks)
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $log(\textcolor{brown}{\theta_{c, w_i}})$ for all words $w_i$ and both classes ($c \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial event model and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [4]:

def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features (words), calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    ### YOUR CODE HERE...
    
    splitt = np.hsplit(data,[1])
    spam_or_ham = splitt[0]
    features = splitt[1]
        
    
    
    
    
    new_features = np.transpose(features)
    new_spam = np.transpose(spam_or_ham)
    
    number_of_emails = new_spam[0].size
    number_of_features = features[0].size
    
    results = np.zeros((2,54))
    results[1][0] = 1
    
    spam_count = 0 #1
    ham_count = 0 #0
    
    for i in range(number_of_features):
            for j in range(number_of_emails):
                if(new_features[i][j]==1):
                    if(new_spam[0][j]==0):
                        ham_count=ham_count+1
                    else:
                        spam_count=spam_count+1
            results[0][i] = ham_count
            results[1][i] = spam_count
            
            spam_count = 0 #1
            ham_count = 0 #0
            
            
            
    #laplace smoothing
    for i in range(2):
            for j in range(number_of_features):
                if(results[i][j]==0):
                    results[i][j] = alpha
                    
                    
    probabilities = np.divide(results,number_of_emails)
    
    
    for i in range(2):
            for j in range(number_of_features):
                    probabilities[i][j] = np.log(probabilities[i][j])
    
  
    return probabilities






In [5]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)


[[-2.24431618 -2.81341072 -1.77785656 -6.90775528 -1.97328135 -2.76462055
  -4.42284863 -3.21887582 -2.93746337 -2.29263476 -3.29683737 -1.33560125
  -2.52572864 -3.61191841 -4.50986001 -2.81341072 -2.7488722  -2.52572864
  -1.07002483 -4.42284863 -1.56542103 -5.29831737 -4.13516656 -4.42284863
  -1.46533757 -1.76609172 -1.81400508 -2.35387839 -2.52572864 -2.40794561
  -2.79688141 -3.05760768 -2.70306266 -3.03655427 -2.3644605  -2.28278247
  -1.74296931 -4.50986001 -2.67364877 -2.81341072 -3.61191841 -2.59026717
  -2.7488722  -2.79688141 -1.71479843 -2.26336438 -4.50986001 -3.32423634
  -2.18925641 -1.08175517 -2.41911891 -1.73160555 -2.76462055 -2.95651156]
 [-1.92414866 -2.12026354 -1.46533757 -5.11599581 -1.37436579 -1.83258146
  -1.85150947 -1.97328135 -2.16282315 -1.69826913 -2.16282315 -1.41881755
  -2.08747371 -3.03655427 -2.91877123 -1.51412773 -1.77785656 -2.0024805
  -1.0584305  -2.44184716 -1.15836229 -4.01738352 -2.07147337 -1.86433016
  -4.50986001 -4.82831374 -6.90775528 

## Part  C: Classify e-mails (30 marks)

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

Use your `predict()` function to classify the messages of your **training data set** `training_spam`. Compute the accuracy of your algorithm _in the training set_ by comparing your predictions to the true class values. Accuracy is simply defined as the proportion of true predictions made by your classifier. Store the accuracy of your naïve Bayes algorithm in a variable called `training_set_accuracy`.

In [8]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    number_of_emails = np.transpose(new_data)[0].size
    number_of_features = new_data[0].size
    maxfinder = np.zeros((2, number_of_emails))
    ######
    for i in range(number_of_emails):
        maxfinder[0][i] =  maxfinder[0][i] + log_class_priors[0]
        maxfinder[1][i] =  maxfinder[1][i] + log_class_priors[1]
    
    classpredictions = np.zeros(number_of_emails)
    
    
    for i in range(number_of_emails):
        for j in range(number_of_features):
            if(new_data[i][j]==1):
                maxfinder[0][i] =  maxfinder[0][i] + log_class_conditional_likelihoods[0][j]
                maxfinder[1][i] =  maxfinder[1][i] + log_class_conditional_likelihoods[1][j]
    
    
    
    for i in range(number_of_emails):
        if(maxfinder[0][i]<maxfinder[1][i]):
            classpredictions[i] = 1
        
        

    
    return classpredictions

def accuracy(y_predictions, y_true):
    """
    Calculate the accuracy.
    
    :param y_predictions: a one-dimensional numpy array of predicted classes (0s and 1s).
    :param y_true: a one-dimensional numpy array of true classes (0s and 1s).
    
    :return acc: a float between 0 and 1 
    """
    ### YOUR CODE HERE...
    number_of_emails = y_predictions.size
    correct_counter = 0
    
    for i in range(number_of_emails):
        if(y_predictions[i] == y_true[i]):
            correct_counter = correct_counter + 1
    
    acc = correct_counter/number_of_emails
    print(acc)
    return acc
    
# training_set_accuracy = ...

In [9]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))
       
# Check accuracy function
true_classes = training_spam[:, 0]
training_set_accuracy = accuracy(class_predictions, true_classes)
assert(isinstance(training_set_accuracy, float))
assert(0 <= training_set_accuracy <= 1)


0.872


## Part D: Classifying previously unseen data (10 marks).
The following cell loads a new set of 500 messages.

In [10]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",")
print("Shape of the testing spam data set:", testing_spam.shape)
testing_spam

Shape of the testing spam data set: (500, 55)


array([[1., 0., 0., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Use the naïve Bayes algorithm that you trained on `training_spam` in order to classify all messages in the `testing_spam` data set. Store the resulting accuracy in a variable called `testing_set_accuracy`.

In [11]:

class_priors = estimate_log_class_priors(testing_spam)
cond_likelihoods = estimate_log_class_conditional_likelihoods(testing_spam,alpha = 1.0)
results = testing_spam[:, 0]
class_predictions = predict(testing_spam[:, 1:], class_priors, cond_likelihoods)

testing_set_accuracy = accuracy(class_predictions,results)





0.838


In [10]:
# This is a test cell. Do not delete or change.

## Part E: Discussion (not marked).


In 3 sentences or less: Compare the accuracy of your classifier on the training set and on the test set. Are they different? If yes, how do you explain the difference?

YOUR ANSWER HERE



In [ ]:
'''We have a train accuracy of 0.889 and an accuracy of 0.85 for the test set. Although the difference is not too high
this can be explained by the fact that we are using a model which was created using the training data set, therefore,
as test data is data unseen by the model, it is more likely that the predictions are more accurate for the training set.'''
